# Introduction to NLP in TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences of text or speech.)

Another common term for NLP problems is sequence to sequence problems (seq2seq0).

In [1]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-13efc026-fe85-f51d-e6fa-c575bd30da6c)


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-10-28 16:44:20--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-10-28 16:44:20 (53.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get a text dataset

The dataset we are going to be using is Kaggel's introduction to NLP dataset (text samples of Tweets as disaster or not disaster).

See the original source here: https://www.kaggle.com/c/nlp-getting-started

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2021-10-28 16:44:23--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 74.125.204.128, 64.233.189.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2021-10-28 16:44:23 (90.6 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
# Unzip data
unzip_data('nlp_getting_started.zip')

## Visualizing a text dataset

To visualize our text sample , we first have to read them in, one way to do so would be to use Python: 

But I prefer to get visual staright away.

So another way to do this is to use pandas.

In [6]:
import pandas as pd

In [7]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
train_df['text'][1]

'Forest fire near La Ronge Sask. Canada'

In [9]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [10]:
# What does the test dataframe looks like
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [11]:
# How many example of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [12]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [13]:
import random
random_index = random.randint(0, len(train_df) - 5) # create random indexes not higher than the total number samples
for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f'Target: {target}, (real disaster)' if target > 0 else '{not real disaster}')
  print(f'Text: \n{text}\n')
  print('-----\n')

{not real disaster}
Text: 
What. The. Fuck. https://t.co/Nv7rK63Pgc

-----

Target: 1, (real disaster)
Text: 
#LoMasVisto THOUSANDS OF HIPSTERS FEARED LOST: Giant Sinkhole Devours Brooklyn Intersectio... http://t.co/qwtk1b2fMC #CadenaDeSeguidores

-----

Target: 1, (real disaster)
Text: 
@AlvinNelson07 A train isn't made to withstand collisions! Immediate derailment. It's totally fucked.

-----

{not real disaster}
Text: 
Love skiing

-----

{not real disaster}
Text: 
We just happened to get on the same road right behind the buses I'm dead serious

-----



### Split data into training and validation sets

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [16]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [17]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

When dealing with a text problem, one of the first thing you have to do before you build a model is , to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenaziation - 
* Embedding - 

### Text Vectorization (tokenization)

In [18]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [19]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [20]:
# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # How many words in the vocabluary (automatically add <00V>)
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None, # create groups of n-words?,
                                    output_mode='int', # how to map tokens to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be.,
                                    pad_to_max_tokens=False,
                                    ) 

In [21]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [22]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [23]:
# Setup text vectorization variables
max_vocab_length = 10000
max_length = 15 # max length our sequences will be (e.g. how many words from tweets does a model see)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [24]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [25]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

In [26]:
# Create a sample sentence and tokenize it
sample_sentence = 'There is a flood in my street!'
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [27]:
# Choose random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f'original text:\n{random_sentence}\n\nVectorized version:')
text_vectorizer([random_sentence])

original text:
Cascada - Evacuate The Dancefloor (Official Video) https://t.co/OHCx3y8l4s via @YouTube

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  1, 279,   2,   1, 839,  72,   1,  49, 114,   0,   0,   0,   0,
          0,   0]])>

In [28]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f'Number of words in vocab: {len(words_in_vocab)}')
print(f'5 most common words: {top_5_words}')
print(f'5 least common words: {bottom_5_words}')

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embeddingusing an Embedding Layer

To make our embedding we are going to use Tensorflow's Embedding layers.

The parameters we care most about for our embedding layser:
* `input_dim` - the size of our vocabluary
* `output_dim` - the size of output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* `input_lenght` - length of sequences being passes to the embedding layer

In [29]:
from tensorflow.keras import layers

In [30]:
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer='uniform',
                             input_length=max_length)

In [31]:
embedding

In [32]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f'Original text:\n{random_sentence}\n\nEmbedded version:')

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Devastation: coming to a @Target and find the @Starbucks closed ?? #momneedscoffee #asap #iwontmakeit

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03313871,  0.01897602,  0.0351136 , ...,  0.0496209 ,
         -0.00155717, -0.03285117],
        [ 0.01636987, -0.04256427, -0.03251853, ...,  0.01121984,
         -0.0487914 , -0.01903006],
        [ 0.00465488, -0.01260176,  0.03700188, ..., -0.01859603,
         -0.02317901,  0.04632098],
        ...,
        [-0.04571028,  0.01214402, -0.02663195, ...,  0.01795422,
          0.04534959, -0.00791057],
        [ 0.04494543,  0.00612792,  0.04754272, ..., -0.04387289,
         -0.03155122,  0.01717368],
        [ 0.04494543,  0.00612792,  0.04754272, ..., -0.04387289,
         -0.03155122,  0.01717368]]], dtype=float32)>

In [33]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.03313871,  0.01897602,  0.0351136 ,  0.0104239 , -0.01501421,
        -0.03216878,  0.00827909, -0.02668587,  0.00087991,  0.01105351,
         0.00859736,  0.02399964, -0.04864323, -0.04765281,  0.02899996,
        -0.02918972,  0.03106386,  0.00683875,  0.03180863,  0.0348482 ,
        -0.02428819,  0.01328609,  0.01929427, -0.00324321, -0.01575433,
         0.0321769 ,  0.02106461,  0.04549298,  0.00185337, -0.01134077,
        -0.01564895, -0.02948715,  0.0095762 ,  0.02480462, -0.03499284,
         0.03414121,  0.03984198,  0.04392072,  0.02302325,  0.03151136,
        -0.00135468, -0.03546492, -0.0093764 , -0.03807511, -0.04791329,
        -0.04976351,  0.03297773,  0.04085939,  0.04293844, -0.03244939,
        -0.01574811, -0.01920158,  0.00126714, -0.03140891,  0.00770028,
         0.03673425, -0.02306554,  0.00583789,  0.01398523, -0.01026446,
         0.03690277, -0.04908544,  0.04391931,  0.00715343, -0.04434158,
  

## Modeling a text dataset (running a series of experiments)

No we have got a way to turn our text sequences into number, it's time to start building a series of modelling experiments.

We will start with a baseline and move on from 

* Model 0: Naiave Bayes (baseline), this is from Sklearn ML
map:

* Model 1: 

* Model 2: LSTM (RNN)

* Model 3: GRU model (RNN)

* Model 4: Bidirectional - LSTM model(RNN)

* Model 5: 1D Convolutional Neural Network (CNN)

* Model 6: TensorFlow HUb Pretrained Feature Extractor (using tranfer learning for NLP)

* Model 7: Same as model 6 with 10% of training data


How are we going to approach all of these?

Use the standard steps in modelling with tensorflow:

* Create a Model
*

### Model 9: Getting a baseline

To create our baseline we will use Sklearn's Multinomial Naive Bayes using the TD-IDF farmula to convert our words into numbers.

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [35]:
# Create tokenization and modeling pipeline
model_0 = Pipeline([
  ('tfidf', TfidfVectorizer()), # convert words to numbers
  ('clf', MultinomialNB()) # model the text            
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [36]:
# Evalutate the model
baseline_score = model_0.score(val_sentences, val_labels)
print(f'Our baseline mocel achieves an accurac of: {baseline_score*100:.2f}%')

Our baseline mocel achieves an accurac of: 79.27%


In [37]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [38]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evalutaion function for our model experiments

We could evaluate all of our model's predictions with different metrics every time, however , this will be combersome and easily be fixed with a function.

Let's create one to compare our model's predictions with the truth lables using the following
* Accuracy
* Precision
* Recall
* F1-score

In [39]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [40]:
def calculate_results(y_true, y_pred):

  # Calcuate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100

  # Calculate model precision, recall and f1-score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
  model_results = {'accuracy': model_accuracy,
                   'precision': model_precision,
                   'recall': model_recall,
                   'f1': model_f1}

  return model_results               


In [41]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [42]:
from helper_functions import calculate_results

In [43]:
print(calculate_results(y_true=val_labels,
                        y_pred=baseline_preds))

{'accuracy': 79.26509186351706, 'precision': 0.8111390004213173, 'recall': 0.7926509186351706, 'f1': 0.7862189758049549}


### Model 1: A simple dense model

In [44]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = 'model_logs'

In [45]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-d string
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector
outputs = layers.Dense(1, activation='sigmoid')(x) # Create the output layer, want binary outputs so use sigmoid activation function
model_1 = tf.keras.Model(inputs, outputs, name='model_1_dense')

In [46]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [47]:
# Compile the model
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [48]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name='model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20211028-164431
Epoch 1/5
215/215 [==============================] - 5s 9ms/step - loss: 0.6103 - accuracy: 0.6955 - val_loss: 0.5335 - val_accuracy: 0.7664
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.4401 - accuracy: 0.8202 - val_loss: 0.4702 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3460 - accuracy: 0.8619 - val_loss: 0.4560 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2837 - accuracy: 0.8907 - val_loss: 0.4629 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2367 - accuracy: 0.9127 - val_loss: 0.4755 - val_accuracy: 0.7874


In [49]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 0.4755 - accuracy: 0.7874


[0.47551077604293823, 0.787401556968689]

In [50]:
# Make some prediction and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

(762, 1)

In [51]:
model_1_pred_probs[0]

array([0.4441704], dtype=float32)

In [52]:
model_1_pred_probs[:10]

array([[0.4441704 ],
       [0.7499537 ],
       [0.9977877 ],
       [0.12964597],
       [0.09317681],
       [0.9477025 ],
       [0.90220547],
       [0.9939645 ],
       [0.9713866 ],
       [0.30075282]], dtype=float32)

In [53]:
val_labels

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In [54]:
# Convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [55]:
# Calcualte our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.74015748031496,
 'f1': 0.7849686345949081,
 'precision': 0.790739069704199,
 'recall': 0.7874015748031497}

In [56]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [57]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

## Visualizing learned embeddings

In [58]:

# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [59]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [60]:
# Get the weight matrix in embedding layer
# (these are the numerical representations of each token in our training data, which have been learned for 5 epocsh)
embed_weights = model_1.get_layer('embedding').get_weights()[0]
embed_weights

array([[ 0.02955481, -0.01101416,  0.0335395 , ..., -0.02798888,
        -0.04748305,  0.00100691],
       [-0.04978676,  0.00731625, -0.03001148, ...,  0.0225991 ,
         0.04197403, -0.01239208],
       [ 0.02904216,  0.01030873, -0.01485948, ...,  0.00112378,
        -0.02365196, -0.02653028],
       ...,
       [ 0.04271208, -0.02424642,  0.01905931, ...,  0.01917282,
        -0.02351713,  0.04168281],
       [-0.05796742, -0.03225105, -0.05580724, ...,  0.02890195,
         0.00658832, -0.07311984],
       [-0.08446391, -0.0861695 , -0.03354079, ...,  0.107513  ,
        -0.05915274, -0.05204869]], dtype=float32)

In [61]:
print(embed_weights.shape) # same size as vocab and embedding dim (output_dim of our embedding layer)

(10000, 128)


In [62]:
embed_weights

array([[ 0.02955481, -0.01101416,  0.0335395 , ..., -0.02798888,
        -0.04748305,  0.00100691],
       [-0.04978676,  0.00731625, -0.03001148, ...,  0.0225991 ,
         0.04197403, -0.01239208],
       [ 0.02904216,  0.01030873, -0.01485948, ...,  0.00112378,
        -0.02365196, -0.02653028],
       ...,
       [ 0.04271208, -0.02424642,  0.01905931, ...,  0.01917282,
        -0.02351713,  0.04168281],
       [-0.05796742, -0.03225105, -0.05580724, ...,  0.02890195,
         0.00658832, -0.07311984],
       [-0.08446391, -0.0861695 , -0.03354079, ...,  0.107513  ,
        -0.05915274, -0.05204869]], dtype=float32)

Now we have got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so, TensorFlow has a 

In [63]:
import io

In [64]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [65]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Networks (RNN's)

RNN's are useful for sequence data.

The premis of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

> **Resources**: If you want an overview of the internals of a recurrent neural network, see the following:
- MIT's lectures : https://www.youtube.com/watch?v=5tvmMX8r_OM&list=PLtBw6njQRU-rwp5__7C0oIVt26ZgjG9NI

- Chris Ola's intro ot LSTMs: 

- Andrej Karpathy's the unreasonable effectivenss of recurrent neural networks:


### Model 2: LSTM

LSTM - Long Short Term Memory (one of the most popular LSTM cells)

Our structure of an RNN typically looks like this:

```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Ouput (label probability)

In [66]:
# Create an LSTM model

from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
# x = layers.LSTM(units=64, return_sequences=True)(x) # When you are stacking RNN cells, you need to return_sequences=True
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation='relu')(x)
# print(x.shape)
outputs =layers.Dense(1, activation='sigmoid')(x)
model_2 = tf.keras.Model(inputs, outputs, name='model_2_LSTM')

In [67]:
# Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
_________________________________________________________________


In [68]:
# Compile the model
model_2.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [69]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_2_LSTM')])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20211028-164444
Epoch 1/5
215/215 [==============================] - 7s 14ms/step - loss: 0.2192 - accuracy: 0.9221 - val_loss: 0.6133 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1548 - accuracy: 0.9403 - val_loss: 0.6484 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1272 - accuracy: 0.9502 - val_loss: 0.6753 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1080 - accuracy: 0.9600 - val_loss: 0.6752 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0875 - accuracy: 0.9669 - val_loss: 1.0776 - val_accuracy: 0.7717


In [70]:
# Make predictions with LSTM Model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[1.5731885e-03],
       [6.1019373e-01],
       [9.9977893e-01],
       [5.7374584e-03],
       [3.3642238e-04],
       [9.9923611e-01],
       [8.0802202e-01],
       [9.9986267e-01],
       [9.9975950e-01],
       [6.7366093e-01]], dtype=float32)

In [71]:
# Convert model 2 pred probs to labels
model_2_pred = tf.squeeze(tf.round(model_2_pred_probs))
model_2_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [72]:
# Calculate model 2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_pred)

model_2_results

{'accuracy': 77.16535433070865,
 'f1': 0.7694523344399302,
 'precision': 0.773559668677779,
 'recall': 0.7716535433070866}

In [73]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 3: GRU

Another popular and effective RNN component is the GRU or gated recurrent unit:

The GRU cells has similar features to an LSTM cell but has less parameters.

In [84]:
# Build an RNN using the GRU cells
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
x = layers.GRU(64, return_sequences=True)(x)
# print(x.shape)
# x = layers.LSTM(42, return_sequences=True)(x)
# print(x.shape)
# x = layers.GRU(99)(x)
# print(x.shape)
# x = layers.Dense(64, activation='relu')(x)
# x = layers.GlobalAveragePooling1D()(x)
x = layers.GRU(64)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name='model_3_GRU')

In [85]:
# Get a summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
gru_13 (GRU)                 (None, 15, 64)            37248     
_________________________________________________________________
gru_14 (GRU)                 (None, 64)                24960     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 65        
Total params: 1,342,273
Trainable params: 1,342,273
Non-trainable params: 0
_____________________________________________

In [88]:
# Compile the model
model_3.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [89]:
# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name='model_3_GRU')])

Saving TensorBoard log files to: model_logs/model_3_GRU/20211028-165831
Epoch 1/5
215/215 [==============================] - 8s 21ms/step - loss: 0.1496 - accuracy: 0.9425 - val_loss: 0.7498 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0842 - accuracy: 0.9696 - val_loss: 1.0597 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0721 - accuracy: 0.9734 - val_loss: 0.8503 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0628 - accuracy: 0.9747 - val_loss: 0.9883 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0556 - accuracy: 0.9771 - val_loss: 0.8784 - val_accuracy: 0.7651


In [94]:
# Make some predictions with our GRU model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[1.2086498e-01],
       [8.7103182e-01],
       [9.9603623e-01],
       [8.9281917e-02],
       [5.5703073e-04],
       [9.9515069e-01],
       [1.3767326e-01],
       [9.9929559e-01],
       [9.9864882e-01],
       [4.0735644e-01]], dtype=float32)

In [95]:
# Convert model 3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 0.], dtype=float32)>

In [96]:
# Calcualte model 3 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 76.50918635170603,
 'f1': 0.7632813381351602,
 'precision': 0.766006698423534,
 'recall': 0.7650918635170604}

### Model 4: Bidirectional RNN

Normal RNN's go from left to right (just like you'd read an English sentence) however bidirectional RNN's goes from right to left and also left to right.

In [104]:
# Build a bidirectional RNN in TensorFlow
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_4 = tf.keras.Model(inputs, outputs, name='model_4_bidirectional')

In [105]:
# Get a summary
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 129       
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________


In [106]:
# Compile model
model_4.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [107]:
# Fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_4_bidirectional')])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20211028-172100
Epoch 1/5
215/215 [==============================] - 8s 21ms/step - loss: 0.1098 - accuracy: 0.9650 - val_loss: 0.9295 - val_accuracy: 0.7651
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0549 - accuracy: 0.9756 - val_loss: 1.1174 - val_accuracy: 0.7730
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0482 - accuracy: 0.9793 - val_loss: 1.2210 - val_accuracy: 0.7612
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0432 - accuracy: 0.9801 - val_loss: 1.5017 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0434 - accuracy: 0.9806 - val_loss: 1.4039 - val_accuracy: 0.7690


In [109]:
# Make predictionas with our bidirectional model
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[4.5147070e-04],
       [6.0625076e-01],
       [9.9987400e-01],
       [7.7414781e-02],
       [1.3862204e-05],
       [9.9753201e-01],
       [5.7186313e-02],
       [9.9996722e-01],
       [9.9992263e-01],
       [6.4514476e-01]], dtype=float32)

In [110]:
# Convert pred probs to pred labels 
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1.], dtype=float32)>

In [112]:
# Calculate the results of our bidirectional model
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=model_4_preds)
model_4_results

{'accuracy': 76.9028871391076,
 'f1': 0.7660901868079131,
 'precision': 0.772410916855732,
 'recall': 0.7690288713910761}

In [113]:
model_3_results

{'accuracy': 76.50918635170603,
 'f1': 0.7632813381351602,
 'precision': 0.766006698423534,
 'recall': 0.7650918635170604}

## Convolutional Neural Networks for Text (and other types of sequences)

We have used CNNs for images but images are typically 2D (height x Width)... however, our text data is 1D.

Previously we have used Conv2D for our image data but now we are going to use Conv1D.

The typical structure of a Conv1D model for sequences , in our case , text:

```
Inputs (text) -> Tokenization -> Embedding -> Layer(s) (typically Conv1D + Pooling) -> Output (class probabilities)```

### Model 5: Conv1D

For different explainations of parameters see:
* https://poloclub.github.io/cnn-explainer/
* Difference between `same` and `valid` padding: https://stackoverflow.com/questions/37674306/what-is-the-difference-between-same-and-valid-padding-in-tf-nn-max-pool-of-t#:~:text=VALID%20%3A%20Don't%20apply%20any,filter%20and%20stride%20you%20specified.

In [114]:
# Test out our embeddingl layer, Conv1D layer and max pooling
embedding_test = embedding(text_vectorizer(['this is a test sentences'])) # turn target sequences into embedding
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5, # this is also referred to as an ngram of 5 (meaning it looks at 5 words at a time)
                        activation='relu',
                        padding='valid')
conv_1d_output = conv_1d(embedding_test) # pass test embedding through conv1d layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # equivalent to "get the most important feature with the highest value"

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [115]:
embedding_test = embedding(text_vectorizer(['this is a test sentences'])) # turn target sequences into embedding
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5,
                        activation='relu',
                        padding='same')
conv_1d_output = conv_1d(embedding_test) # pass test embedding through conv1d layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # equivalent to "get the most important feature with the highest value"

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 15, 32]), TensorShape([1, 32]))

In [119]:
# embedding_test

In [120]:
# conv_1d_output

In [ ]:
# max_pool_output